# Packing circuits

This notebook looks at some examples of packing circuits to reduce their ancilla costs. In particular in these examples this problem is solved via the calculation of minimum vertex covers, which is done by not allowing 'evicted gates'.

First, import the modules we need.

In [1]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket_dqc.networks import NISQNetwork
from pytket_dqc.allocators import HypergraphPartitioning, Annealing
from pytket_dqc.circuits import HypergraphCircuit, BipartiteCircuit
from pytket_dqc.utils import ebit_memory_required
from pytket.transform import Transform

## Trivial examples

### Two CZs

In [2]:
circuit = Circuit(2)
circuit.CZ(0,1)
circuit.CZ(0,1)
network = NISQNetwork(
    [[0,1]],
    {0: [0], 1: [1]}
)
render_circuit_jupyter(circuit)

dist_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
distribution = distributor.allocate(dist_circ, network)

In [3]:
bp_circuit = BipartiteCircuit(circuit, distribution.placement)
render_circuit_jupyter(bp_circuit.packed_circuit)

In [4]:
print("Ebit memory required per server:")
for server, mem in ebit_memory_required(bp_circuit.packed_circuit).items():
    print(f"\tServer {server} requires {mem} memory qubits")

Ebit memory required per server:
	Server 0 requires 1 memory qubits
	Server 1 requires 0 memory qubits


### Two CZs (opposite controls)

In [5]:
circuit = Circuit(2)
circuit.CZ(0,1)
circuit.CZ(1,0)
network = NISQNetwork(
    [[0,1]],
    {0: [0], 1: [1]}
)
render_circuit_jupyter(circuit)

dist_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
distribution = distributor.allocate(dist_circ, network)

In [6]:
bp_circuit = BipartiteCircuit(circuit, distribution.placement)
render_circuit_jupyter(bp_circuit.packed_circuit)

In [7]:
print("Ebit memory required per server:")
for server, mem in ebit_memory_required(bp_circuit.packed_circuit).items():
    print(f"\tServer {server} requires {mem} memory qubits")

Ebit memory required per server:
	Server 0 requires 1 memory qubits
	Server 1 requires 0 memory qubits


### Two CZs with X in between

In [8]:
circuit = Circuit(2)
circuit.CZ(0,1)
circuit.Rx(1, 0)
circuit.Rx(1, 1)
circuit.CZ(1,0)
network = NISQNetwork(
    [[0,1]],
    {0: [0], 1: [1]}
)
render_circuit_jupyter(circuit)

dist_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
distribution = distributor.allocate(dist_circ, network)

In [9]:
bp_circuit = BipartiteCircuit(circuit, distribution.placement)
render_circuit_jupyter(bp_circuit.packed_circuit)

In [10]:
print("Ebit memory required per server:")
for server, mem in ebit_memory_required(bp_circuit.packed_circuit).items():
    print(f"\tServer {server} requires {mem} memory qubits")

Ebit memory required per server:
	Server 0 requires 1 memory qubits
	Server 1 requires 0 memory qubits


### Two CZs with non-packable in between

In [11]:
circuit = Circuit(2)
circuit.CZ(0,1)
circuit.Rx(0.5, 0)
circuit.Rx(0.5, 1)
circuit.CZ(1,0)
network = NISQNetwork(
    [[0,1]],
    {0: [0], 1: [1]}
)
render_circuit_jupyter(circuit)

dist_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
distribution = distributor.allocate(dist_circ, network)

In [12]:
bp_circuit = BipartiteCircuit(circuit, distribution.placement)
render_circuit_jupyter(bp_circuit.packed_circuit)

In [13]:
print("Ebit memory required per server:")
for server, mem in ebit_memory_required(bp_circuit.packed_circuit).items():
    print(f"\tServer {server} requires {mem} memory qubits")

Ebit memory required per server:
	Server 0 requires 1 memory qubits
	Server 1 requires 0 memory qubits


### Simple CZ cycle

In [14]:
circuit = Circuit(3)
circuit.CZ(0, 1)
circuit.CZ(1, 2)
circuit.CZ(2, 0)
network = NISQNetwork(
    [[0,1], [1, 2]],
    {0: [0], 1: [1], 2:[2]}
)
render_circuit_jupyter(circuit)

dist_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
distribution = distributor.allocate(dist_circ, network)

In [15]:
bp_circuit = BipartiteCircuit(circuit, distribution.placement)
render_circuit_jupyter(bp_circuit.packed_circuit)

In [16]:
print("Ebit memory required per server:")
for server, mem in ebit_memory_required(bp_circuit.packed_circuit).items():
    print(f"\tServer {server} requires {mem} memory qubits")

Ebit memory required per server:
	Server 0 requires 2 memory qubits
	Server 1 requires 1 memory qubits
	Server 2 requires 0 memory qubits


### Complicated circuit

In [17]:
circuit = Circuit(6)
circuit.CZ(0, 3).CZ(2, 3).CZ(2, 4).H(2).CZ(2, 5).CZ(2, 4).CZ(0, 3).CZ(0, 4).H(3).CZ(0, 5).X(0).CZ(0, 3)
Transform.RebaseToQuil().apply(circuit)
network = NISQNetwork([[0,1]], {0:[0,1,2], 1:[3,4,5]})
render_circuit_jupyter(circuit)

dist_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
distribution = distributor.allocate(dist_circ, network)

In [18]:
bp_circuit = BipartiteCircuit(circuit, distribution.placement)
render_circuit_jupyter(bp_circuit.packed_circuit)

In [19]:
print("Ebit memory required per server:")
for server, mem in ebit_memory_required(bp_circuit.packed_circuit).items():
    print(f"\tServer {server} requires {mem} memory qubits")

Ebit memory required per server:
	Server 0 requires 2 memory qubits
	Server 1 requires 0 memory qubits
